In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import unicodedata

In [2]:
def cambio_enti(s):
    llista = s.lower().split(' ')
    n_llista = [i.capitalize() if i != 'de' and i != 'del' else i for i in llista ]
    s = ' '.join(n_llista)
    if '-' in s:
        llista = s.lower().split('-')
        n_llista = [i.capitalize() if i != 'de' and i != 'del' else i for i in llista ]
        s = ' '.join(n_llista)
    if '(la)' in s:
        s = 'La '+s.replace('(la)','').strip()
    if '(el)' in s:
        s = 'El '+s.replace('(el)','').strip()
    if '(los)' in s:
        s = 'Los '+s.replace('(los)','').strip()
    if '(las)' in s:
        s = 'Las '+s.replace('(las)','').strip()   

    if 'de La ' in s:
        s = s.replace('de La ', 'de la ')
    if 'de Las ' in s:
        s = s.replace('de Las ', 'de las ')
    if 'de Los ' in s:
        s = s.replace('de Los ', 'de los ')
    if ' Y ' in s:
        s = s.replace(' Y ', ' y ')
    if ' O ' in s:
        s = s.replace(' O ', ' o ')
    return s
def scrap_wiki_pop(x):
    provincia, municipio, entidad = x
    n_e = cambio_enti(entidad)
    try:
        URL = f"https://es.wikipedia.org/wiki/{n_e.replace(' ','_')}"
        #print(URL)
        r = requests.get(URL)
        soup = BeautifulSoup(r.content, 'lxml') 
        bodies = soup.findAll('tbody')
        a = False
        for e in bodies:
            if 'Población' in e.text:
                r_tabla = e.findAll('tr')
                a=True
                break

        if not a: 
            URL = f"https://es.wikipedia.org/wiki/{n_e.replace(' ','-')}_({municipio.replace(' ','_')})"
            r = requests.get(URL)
            soup = BeautifulSoup(r.content, 'lxml') 
            bodies = soup.findAll('tbody')
            for e in bodies:
                if 'Población' in e.text: a=True
        
        if not a: 
            URL = f"https://es.wikipedia.org/wiki/{n_e.replace(' ','_')}_({municipio.replace(' ','_')})"
            r = requests.get(URL)
            soup = BeautifulSoup(r.content, 'lxml') 
            bodies = soup.findAll('tbody')
            for e in bodies:
                if 'Población' in e.text:
                    a=True
        if not a: 
            URL = f"https://es.wikipedia.org/wiki/{n_e.replace(' ','_')}_({provincia})"
            r = requests.get(URL)
            soup = BeautifulSoup(r.content, 'lxml') 
            bodies = soup.findAll('tbody')
            for e in bodies:
                if 'Población' in e.text:
                    a=True
        table = soup.find('table', {'class': 'infobox geography vcard'})
        rows = table.findAll('tr')
        for tr in rows:
            cols = tr.findAll('td')
            if len(cols) > 0 and 'hab' in cols[0].text and 'km' not in cols[0].text:
                pob = unicodedata.normalize("NFKD", str(cols[0].get_text(strip=True)))
                
        return pob
    except:
        # print(n_e)
        # i += 1
        # if i > 1000:
        #     i = 0
        #     print('ok')
        return np.nan

In [3]:
scrap_wiki_pop(['Girona','Berga','Berga'])

'16 682 hab.(2021)'

In [4]:
df = pd.read_csv('../Data/castilla_leon/main.csv', index_col=['Unnamed: 0'])

In [5]:
df['population'] = df.apply(lambda x: scrap_wiki_pop((x['Provincia'],x['Municipio'],x['Entidad Singular de Población'])), axis=1)

In [ ]:
df.head()